**Section	A:	Problem	Statement	–	Enhancing	Neural	Network	Performance	with Particle	Swarm	Optimization**	


In [1]:
# I coose Predicting	secondary	school	student	performance	using	a	double	particle	swarm optimization-based	categorical	boosting	model
# link - (https://www.sciencedirect.com/science/article/abs/pii/S0952197623008333)

In [2]:
# Use Python's requests Module to download the file directly in Jupyter Notebook
import requests

# URL of the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip"

# Send HTTP request
response = requests.get(url)

# Save the content to a zip file
with open("student-performance.zip", "wb") as file:
    file.write(response.content)

print("Download complete!")

Download complete!


In [3]:
# Extract the ZIP File
import zipfile

# Extract the dataset
with zipfile.ZipFile("student-performance.zip", 'r') as zip_ref:
    zip_ref.extractall("student_performance")

print("Extraction complete!")



Extraction complete!


In [4]:
#  Load the Dataset
import pandas as pd

# Load the student performance dataset
df = pd.read_csv("student_performance/student-mat.csv", sep=";")

# Display the first few rows
df.head()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [5]:
!pip install pyswarm



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from pyswarm import pso

In [7]:
# Import Libraries
import random
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from pyswarm import pso  # PSO optimization library

# Suppress warnings
warnings.filterwarnings("ignore")


In [8]:
# PSO-NN Model for Hyperparameter Optimization

import random
import warnings
from sklearn.neural_network import MLPClassifier  # Import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



# Load dataset (modify based on your actual data)
df = pd.read_csv("student_performance/student-mat.csv", sep=";")
df_encoded = pd.get_dummies(df, drop_first=True)  # One-hot encoding categorical columns

X = df_encoded.drop("G3", axis=1)  # Features (all except target column 'G3')
y = df_encoded["G3"]  # Target variable (final grade 'G3')

In [9]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
# PSO Configuration
SWARM_SIZE = 10
DIMENSIONS = 2  # [learning rate, hidden layer size]
NUM_GENERATIONS = 20
W = 0.729  # Inertia weight
C1 = 1.49  # Cognitive weight
C2 = 1.49  # Social weight
MIN_BOUNDARY = [0.0001, 5]  # Minimum learning rate, minimum hidden neurons
MAX_BOUNDARY = [0.1, 100]  # Maximum learning rate, maximum hidden neurons


In [11]:
# Fitness function
def fitness_function(position):
    lr = position[0]
    hidden = int(position[1])  # Convert to integer for hidden layer size
    if hidden <= 0:
        hidden = 1  # Ensure hidden layer size is positive

    model = MLPClassifier(hidden_layer_sizes=(hidden,), learning_rate_init=lr, max_iter=200, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate performance on the test set
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)  # Accuracy as fitness function
    return 1 - acc  # We want to minimize error


In [12]:
# Particle Class (for PSO)
class Particle:
    def __init__(self):
        self.position = [
            random.uniform(MIN_BOUNDARY[0], MAX_BOUNDARY[0]),
            random.uniform(MIN_BOUNDARY[1], MAX_BOUNDARY[1])
        ]
        self.velocity = [random.uniform(-1, 1) for _ in range(DIMENSIONS)]
        self.fitness = fitness_function(self.position)
        self.best_position = list(self.position)
        self.best_fitness = self.fitness
        self.informants = random.sample(range(SWARM_SIZE), 3)
        self.group_best_position = list(self.position)
        self.group_best_fitness = self.fitness

    def update_velocity(self):
        for d in range(DIMENSIONS):
            r1, r2 = random.random(), random.random()
            cognitive = C1 * r1 * (self.best_position[d] - self.position[d])
            social = C2 * r2 * (self.group_best_position[d] - self.position[d])
            self.velocity[d] = W * self.velocity[d] + cognitive + social

    def update_position(self):
        for d in range(DIMENSIONS):
            self.position[d] += self.velocity[d]
            self.position[d] = max(MIN_BOUNDARY[d], min(MAX_BOUNDARY[d], self.position[d]))
        self.fitness = fitness_function(self.position)

    def update_group_best(self, swarm):
        best_informant = min(self.informants, key=lambda i: swarm[i].best_fitness)
        if swarm[best_informant].best_fitness < self.group_best_fitness:
            self.group_best_fitness = swarm[best_informant].best_fitness
            self.group_best_position = list(swarm[best_informant].best_position)

In [13]:
# PSO main loop
swarm = [Particle() for _ in range(SWARM_SIZE)]
global_best = min(swarm, key=lambda p: p.best_fitness)
global_best_position = list(global_best.best_position)
global_best_fitness = global_best.best_fitness

for gen in range(NUM_GENERATIONS):
    for particle in swarm:
        particle.update_group_best(swarm)
        particle.update_velocity()
        particle.update_position()
        if particle.fitness < particle.best_fitness:
            particle.best_fitness = particle.fitness
            particle.best_position = list(particle.position)

    best_particle = min(swarm, key=lambda p: p.best_fitness)
    if best_particle.best_fitness < global_best_fitness:
        global_best_fitness = best_particle.best_fitness
        global_best_position = list(best_particle.best_position)

    print(f"Generation {gen + 1}: Best Accuracy = {1 - global_best_fitness:.4f}")

print("\nPSO Optimization Complete!")
print(f"Best Learning Rate: {global_best_position[0]:.5f}")
print(f"Best Hidden Neurons: {int(global_best_position[1])}")

Generation 1: Best Accuracy = 0.2658
Generation 2: Best Accuracy = 0.3165
Generation 3: Best Accuracy = 0.3165
Generation 4: Best Accuracy = 0.3165
Generation 5: Best Accuracy = 0.3165
Generation 6: Best Accuracy = 0.3165
Generation 7: Best Accuracy = 0.3165
Generation 8: Best Accuracy = 0.3165
Generation 9: Best Accuracy = 0.3165
Generation 10: Best Accuracy = 0.3165
Generation 11: Best Accuracy = 0.3165
Generation 12: Best Accuracy = 0.3165
Generation 13: Best Accuracy = 0.3165
Generation 14: Best Accuracy = 0.3165
Generation 15: Best Accuracy = 0.3165
Generation 16: Best Accuracy = 0.3165
Generation 17: Best Accuracy = 0.3165
Generation 18: Best Accuracy = 0.3165
Generation 19: Best Accuracy = 0.3165
Generation 20: Best Accuracy = 0.3165

PSO Optimization Complete!
Best Learning Rate: 0.10000
Best Hidden Neurons: 95


In [14]:
# Traditionally Optimized Neural Network using Grid Search

#import library

import time
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


In [15]:
# Define parameter grid for grid search
param_grid = {
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [200]
}

# Initialize the model
mlp = MLPClassifier(random_state=42)

In [16]:
# Perform Grid Search
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='accuracy')
start_time = time.time()
grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time


In [17]:
# Print the best parameters and the corresponding accuracy
print("Best parameters from Grid Search:", grid_search.best_params_)
print("Best Accuracy from Grid Search:", grid_search.best_score_)

Best parameters from Grid Search: {'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01, 'max_iter': 200}
Best Accuracy from Grid Search: 0.24365079365079367


In [18]:
# Evaluate on the test set
best_mlp = grid_search.best_estimator_
y_pred_grid = best_mlp.predict(X_test)
grid_accuracy = accuracy_score(y_test, y_pred_grid)
print(f"Test Accuracy from Grid Search: {grid_accuracy:.4f}")
print(f"Grid Search Time: {grid_search_time:.2f} seconds")

Test Accuracy from Grid Search: 0.2658
Grid Search Time: 11.23 seconds


In [19]:
# Final Evaluation

# Display comparison results
print("\n--- Final Results Comparison ---")
print(f"PSO-NN Test Accuracy: {1 - global_best_fitness:.4f}")
print(f"Grid Search Test Accuracy: {grid_accuracy:.4f}")
print(f"PSO-NN Time: {sum([particle.fitness for particle in swarm]):.2f} seconds")
print(f"Grid Search Time: {grid_search_time:.2f} seconds")



--- Final Results Comparison ---
PSO-NN Test Accuracy: 0.3165
Grid Search Test Accuracy: 0.2658
PSO-NN Time: 7.62 seconds
Grid Search Time: 11.23 seconds
